<a href="https://colab.research.google.com/github/Skander28/Models/blob/main/bilstm_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/

'carte visite badra.gdoc'
 Classroom
'Colab Notebooks'
 django
 entreprise-1-20220417T102515Z-001.zip
'Espace citoyen.pdf'
 ez-learning
 maghrebi
'model (1).pth'
 preprocessed_tweets.csv
 Projet_Java_FediBoussaada_LaribiSkander_2A
 Projet_Microservices
 trainer2.pth
'Untitled presentation.gslides'


In [4]:
import pandas as pd
filtered_df = pd.read_csv('/content/drive/MyDrive/preprocessed_tweets.csv')
filtered_df.head()

,Unnamed: 0,id,dialect,tweets,tokens
0,15497,1009754958479151232,1,قليلين ادب ومنافقين اختهم او قريبتهم تعاكس تقو...,"['قليلين', 'ادب', 'ومنافقين', 'اختهم', 'قريبته..."
1,15498,1009794751548313600,1,اليبين متقلبين بالنسبه ليا انا ميليشياوي زمان ...,"['متقلبين', 'ميليشياوي', 'وتوه']"
2,15499,1019989115490787200,1,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...,"['تانيه', 'شاب', 'بيرتاح', 'لبنت', 'مختلفه', '..."
3,15500,1035479791758135168,1,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...,"['رانيا', 'عقليتك', 'متخلفه', 'اولا', 'الانسان..."
4,15501,1035481122921164800,1,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...,"['شكلك', 'متعقده', 'علشان', 'تحبيه', 'ازوج', '..."


In [5]:
filtered_df = filtered_df.dropna()

In [6]:
# Load data
features = filtered_df.tweets.values
labels = pd.get_dummies(filtered_df['dialect']).values

In [7]:
vocab_size = 20000
max_length= 200
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)

In [8]:
import nltk
from collections import Counter
nltk.download('stopwords')
from nltk.corpus import stopwords

# use the stopwords
stop_words = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
all_words = []
for tokens in X:
    all_words.extend(tokens)

stop_words = set(stopwords.words('arabic'))
all_words = [word for word in all_words if word not in stop_words]

word_counts = Counter(all_words)
most_common_words = [word for word, count in word_counts.most_common(500)]

def remove_common_words(tokens):
  new_tokens = [token for token in tokens if token not in most_common_words]
  return new_tokens

X = remove_common_words(X)


X = pad_sequences(X, maxlen=max_length)

In [10]:
# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

In [11]:
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train,y_train)

In [12]:
# Convert data to PyTorch tensors
X_res_, y_res_ = torch.tensor(X_res), torch.tensor(y_res)
X_val, y_val = torch.tensor(X_val), torch.tensor(y_val)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

In [13]:
class DialectDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [14]:
# Create dataloaders for training, validation, and test sets
train_dataset = DialectDataset(X_res, y_res)
val_dataset = DialectDataset(X_val, y_val)
test_dataset = DialectDataset(X_test, y_test)

In [15]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np

vocab_size = 20000
embedding_dim = 100
hidden_dim = 128
output_dim = 4
num_layers = 2
bidirectional = True
lr = 0.01
batch_size = 64
num_epochs = 100
patience = 5

class BiLSTMAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers):
        super(BiLSTMAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=True, dropout=0.5, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.5)
        self.attention = SelfAttention(hidden_dim * 2)
        
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        attention_out, attention_weights = self.attention(lstm_out)
        fc_out = self.fc(attention_out)
        return fc_out, attention_weights

class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, encoder_outputs):
        energy = self.projection(encoder_outputs)
        weights = F.softmax(energy.squeeze(-1), dim=1)
        outputs = (encoder_outputs * weights.unsqueeze(-1)).sum(dim=1)
        return outputs, weights

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BiLSTMAttention(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers).to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01)
criterion = nn.BCEWithLogitsLoss()
counter_nb = 0
for epoch in tqdm(range(24)):
     model.train()
     counter_nb = counter_nb + 1 
     print(counter_nb)
     running_loss = 0.0
     with torch.cuda.device(0):
       for batch in train_dataloader:
         inputs, labels = batch[0].to(device), batch[1].to(device)
         optimizer.zero_grad()
         labels = labels.float()
         outputs, _ = model(inputs)
         loss = criterion(outputs, labels)
         loss.backward()
         optimizer.step()
         running_loss += loss.item()
         preds = torch.sigmoid(outputs) > 0.5
         acc = accuracy_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy())
         #print('Train Accuracy: {:.4f}'.format(acc))
     epoch_loss = running_loss / (len(train_dataloader))

     best_val_loss = np.inf
     patience = 3
     counter = 0
     val_loss = 0.0
     model.eval()
     with torch.no_grad():
         for batch in val_dataloader:
             inputs, labels = batch[0].to(device), batch[1].to(device)
             labels = labels.float()
             outputs, _= model(inputs)
             loss = criterion(outputs, labels)
             val_loss += loss.item()
             val_preds = torch.sigmoid(outputs) > 0.5
             val_acc = accuracy_score(labels.cpu().detach().numpy(), val_preds.cpu().detach().numpy())
             #print('Val Accuracy: {:.4f}'.format(val_acc))
     val_loss = val_loss / (len(val_dataloader))
     # Check if the validation loss has improved
     if val_loss < best_val_loss:
                best_val_loss = val_loss
                counter = 0
     else:
         counter += 1

     # Stop the training process if the validation loss hasn't improved for `patience` epochs
     if counter >= patience:
         break
     print(" epoch loss :", epoch_loss , "| val loss :", val_loss)
     print('Train Accuracy: {:.4f}'.format(acc))
     print('Val Accuracy: {:.4f}'.format(val_acc))

print("Training stopped after epoch", epoch)
        

  0%|          | 0/24 [00:00<?, ?it/s]

1


  4%|▍         | 1/24 [00:16<06:18, 16.45s/it]

 epoch loss : 0.5228401251377598 | val loss : 0.47905563992949635
Train Accuracy: 0.2727
Val Accuracy: 0.5294
2


  8%|▊         | 2/24 [00:32<05:54, 16.11s/it]

...

 epoch loss : 0.09062164422845648 | val loss : 0.7125798168663795
Train Accuracy: 0.9091
Val Accuracy: 0.5294
23


 96%|█████████▌| 23/24 [05:59<00:15, 15.44s/it]

 epoch loss : 0.08368662137498138 | val loss : 0.7590890876375712
Train Accuracy: 0.9318
Val Accuracy: 0.5882
24


100%|██████████| 24/24 [06:14<00:00, 15.60s/it]

 epoch loss : 0.09117864037072787 | val loss : 0.7254087692843034
Train Accuracy: 0.8409
Val Accuracy: 0.5294
Training stopped after epoch 23


In [18]:
from sklearn.metrics import f1_score
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for batch in test_dataloader:
        inputs, labels = batch[0].to(device), batch[1].to(device)
        labels = labels.float()
        outputs, _ = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        y_true.extend(labels.cpu().detach().numpy())
        y_pred.extend(preds.cpu().detach().numpy())
test_acc = accuracy_score(y_true, y_pred)
test_f1 = f1_score(y_true, y_pred, average='macro')
print('Test Accuracy: {:.4f}'.format(test_acc))

Test Accuracy: 0.6283


In [19]:
torch.save(model.state_dict(), 'modeling.pth')